# 3 Generate Predicted Data

This module is used for training the prediction model and generating prediction data.

After running notebooks 1 and 2, run all cells of this notebook to get the predicted dataset to "prediction.csv".
Creates also "midsummers_ts.csv" for visualizations.

In [8]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import datetime, timedelta
from sklearn import model_selection, preprocessing, ensemble
%matplotlib inline

In [3]:
# data parameters
categorical_columns = ["vehicle_type", "location_id", "direction", "weekday", "week"]
dependent_columns = ["hour_{}".format(n) for n in range(1,25)]

# functions
def to_train_format(raw_data):
    """
    Converts categorical fields to dummies (and normalizes the values).
    Returns a DataFrame.
    """
    data = raw_data
    for col in categorical_columns:
        dummies = pd.get_dummies(data[col], prefix=col)
        data = pd.concat([data, dummies], axis=1)
        data = data.drop([col], axis=1)
    return data

def train_test_split(data, test_size=0.10):
    """
    Splits the formatted data into train and test sets.
    Returns X and Y + associated test sets as DataFrames.
    """
    X = data.drop(dependent_columns, axis=1)
    Y = data[dependent_columns]
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size)
    return X_train, X_test, Y_train, Y_test

def train(X_train, Y_train, model):
    """
    Fits the model to the train data.
    Modifies the model parameter.
    """
    model.fit(X_train, Y_train)
    model.features = X_train.columns
    model.targets = Y_train.columns

def evaluate(X_test, Y_test, model):
    """
    Returns the coefficient of determination R^2 of the prediction.
    """
    score = model.score(X_test, Y_test)
    return score

def evaluate_target_period():
    return

def predict(row, model):
    """
    Uses the trained model to predict with the given DataFrame.
    Returns a DataFrame including the predictions.
    """
    row_f = to_train_format(row)
    row_formatted = pd.DataFrame()
    for col in model.features:
        if col in row_f.columns:
            row_formatted[col] = row_f[col]
        else:
            row_formatted[col] = 0
    pred = pd.DataFrame(model.predict(row_formatted))
    pred.columns = model.targets
    return pred

def sample_prediction(sample_data, model):
    """
    Generates the prediction of a sample item.
    Returns a DataFrame containing the original and the 
    predicted values side-by-side.
    """
    sample = sample_data.sample(1)
    result = pd.concat([sample, predict(sample, model)]).fillna("").transpose()
    index = result.columns[0]
    result.columns = [index, "prediction"]
    return result

def generate_prediction_series(
        model,
        dates=["2018-06-18", "2018-06-19", "2018-06-20", "2018-06-21", "2018-06-22", "2018-06-23", "2018-06-24", "2018-06-25", "2018-06-26"], 
        location_ids=[168, 1403, 110],
        vehicle_types=["13 Linja-autot", "12 KAIP", "11 HA-PA"],
    ):
    """
    Generates a predicted data series for the given dates, 
    locations, and vehicle types. Returns a DataFrame.
    """
    generated = pd.DataFrame()
    for date_str in dates:
        date = parser.parse(date_str)
        for location_id in location_ids:
            for vehicle_type in vehicle_types:
                for direction in [1,2]:
                    row = pd.DataFrame()
                    row["year"] = [date.year]
                    row["week"] = [date.isocalendar()[1]]
                    row["weekday"] = [date.weekday()]
                    row["location_id"] = [location_id]
                    row["direction"] = [direction]
                    row["vehicle_type"] = [vehicle_type]
                    #row["temperature"] = [15]
                    #row["rain"] = [0]
                    prediction = predict(row, model)
                    prediction.columns = dependent_columns
                    combined = pd.concat([row, prediction], axis=1)
                    generated = generated.append(combined, ignore_index=True)
    return generated

def get_location_name(location_id):
    """
    Returns the LAM station name of a location_id.
    """
    names = {168: "Askisto", 1403: "Kemijärvi", 110: "Mäntsälä"}
    return names[location_id]

def get_ts_row(data, location_id, year, week, weekday, hour):
    """
    Returns time series formatted row from the data. Includes only Cars & Vans, Trucs and Buses.
    """
    ts_row = pd.DataFrame()
    rows = data[(data.location_id == location_id) & (data.year == year) & (data.week == week) & (data.weekday == weekday)]
    cars_1 = rows[(rows.vehicle_type == "11 HA-PA") & (rows.direction == 1)]["hour_{}".format(hour)].iloc[0]
    cars_2 = rows[(rows.vehicle_type == "11 HA-PA") & (rows.direction == 2)]["hour_{}".format(hour)].iloc[0]
    trucks_1 = rows[(rows.vehicle_type == "12 KAIP") & (rows.direction == 1)]["hour_{}".format(hour)].iloc[0]
    trucks_2 = rows[(rows.vehicle_type == "12 KAIP") & (rows.direction == 2)]["hour_{}".format(hour)].iloc[0]
    buses_1 = rows[(rows.vehicle_type == "13 Linja-autot") & (rows.direction == 1)]["hour_{}".format(hour)].iloc[0]
    buses_2 = rows[(rows.vehicle_type == "13 Linja-autot") & (rows.direction == 2)]["hour_{}".format(hour)].iloc[0]
    date = datetime.strptime("{}-{}-{}".format(year, week, weekday+1), "%Y-%W-%u")
    ts_row["date"] = [date]
    ts_row["hour"] = "{:02}-{:02}".format(hour-1, hour)
    ts_row["LAM Station"] = [get_location_name(location_id)]
    ts_row["Cars and Vans - 1"] = cars_1
    ts_row["Cars and Vans - 2"] = cars_2
    ts_row["Trucks - 1"] = trucks_1
    ts_row["Trucks - 2"] = trucks_2
    ts_row["Buses - 1"] = buses_1
    ts_row["Buses - 2"] = buses_2
    return ts_row

def to_ts_format(data):
    """
    Converts the given dataset (of training or generating format) 
    to time series format. Returns a DataFrame.
    """
    ts_data = pd.DataFrame()
    for location_id in data.location_id.unique():
        for year in data[data.location_id == location_id].year.unique():
            for week in data[(data.location_id == location_id) & (data.year == year)].week.unique():
                for weekday in data[(data.location_id == location_id) & (data.year == year) & (data.week == week)].weekday.unique():
                    for hour in range(1,25):
                        try:
                            row = get_ts_row(data, location_id, year, week, weekday, hour)
                            ts_data = pd.concat([ts_data, row])
                        except (ValueError, IndexError) as error:
                            print("Failed: ", location_id, year, week, weekday, hour)
                            print(error)
    return ts_data.reset_index(drop=True)
    

In [4]:
# split the raw data to train and test sets
raw_data = pd.read_csv("refined_dataset.csv").sort_values(by="date")
raw_data = raw_data.drop(["sum", "location_name", "date"], axis=1)
model_data = raw_data
train_formatted_data = to_train_format(model_data)
X_train, X_test, Y_train, Y_test = train_test_split(train_formatted_data)

# random forest regression model
model = ensemble.RandomForestRegressor()

# train the model
train(X_train, Y_train, model)

# evaluate the model
evaluate(X_test, Y_test, model)

0.97646094238674086

In [29]:
# evaluate how the model performs against the target period in 2017 (fron Thu to Tue around the midsummer)
# NOTE: this data was used for training the model, thus being so accurate
target_period = train_formatted_data[
    (train_formatted_data["year"] == 2017) &
    ((train_formatted_data["week_25"] == 1) & (
        (train_formatted_data["weekday_3"] == 1) |
        (train_formatted_data["weekday_4"] == 1) |
        (train_formatted_data["weekday_5"] == 1) |
        (train_formatted_data["weekday_6"] == 1)
    )) | 
    ((train_formatted_data["week_26"] == True) & (
        (train_formatted_data["weekday_0"] == 1) |
        (train_formatted_data["weekday_1"] == 1)
    ))
]
X_test_target_period = target_period.drop(dependent_columns, axis=1)
Y_test_target_period = target_period[dependent_columns]
evaluate(X_test_target_period, Y_test_target_period, model)

0.99606267838865059

In [ ]:
# evaluate how the model performs against data that was not used in testing

In [9]:
# generate series over the prediction period and store as CSV
prediction_period = [(datetime.today() + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(0, 100)]
generated = generate_prediction_series(model, dates=prediction_period)
generated_ts = to_ts_format(generated)
generated_ts.to_csv("prediction_ts.csv", index=False)

In [6]:
# create also ts formatted DataFrame from historical data for the relevant vehicles.
historical = raw_data[raw_data["vehicle_type"].isin(["13 Linja-autot", "12 KAIP", "11 HA-PA"])]
historical_ts = to_ts_format(historical)
historical_ts.to_csv("historical_ts.csv", index=False)

Failed:  1403 2010 4 6 1
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 2
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 3
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 4
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 5
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 6
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 7
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 8
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 9
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 10
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 11
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 12
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 13
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 14
single positional indexer is out-of-bounds
Failed:  1403 2010 4 6 15
single positional

Failed:  1403 2010 46 6 1
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 2
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 3
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 4
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 5
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 6
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 7
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 8
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 9
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 10
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 11
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 12
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 13
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 14
single positional indexer is out-of-bounds
Failed:  1403 2010 46 6 15
si

Failed:  1403 2011 22 3 1
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 2
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 3
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 4
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 5
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 6
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 7
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 8
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 9
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 10
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 11
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 12
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 13
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 14
single positional indexer is out-of-bounds
Failed:  1403 2011 22 3 15
si

Failed:  1403 2011 44 5 1
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 2
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 3
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 4
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 5
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 6
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 7
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 8
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 9
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 10
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 11
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 12
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 13
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 14
single positional indexer is out-of-bounds
Failed:  1403 2011 44 5 15
si

Failed:  1403 2011 51 6 1
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 2
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 3
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 4
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 5
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 6
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 7
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 8
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 9
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 10
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 11
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 12
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 13
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 14
single positional indexer is out-of-bounds
Failed:  1403 2011 51 6 15
si

...

Failed:  1403 2016 49 1 1
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 2
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 3
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 4
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 5
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 6
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 7
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 8
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 9
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 10
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 11
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 12
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 13
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 14
single positional indexer is out-of-bounds
Failed:  1403 2016 49 1 15
si

Failed:  1403 2017 2 6 1
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 2
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 3
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 4
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 5
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 6
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 7
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 8
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 9
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 10
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 11
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 12
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 13
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 14
single positional indexer is out-of-bounds
Failed:  1403 2017 2 6 15
single positional

Failed:  1403 2017 14 6 1
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 2
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 3
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 4
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 5
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 6
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 7
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 8
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 9
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 10
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 11
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 12
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 13
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 14
single positional indexer is out-of-bounds
Failed:  1403 2017 14 6 15
si

Failed:  1403 2017 22 5 1
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 2
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 3
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 4
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 5
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 6
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 7
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 8
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 9
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 10
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 11
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 12
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 13
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 14
single positional indexer is out-of-bounds
Failed:  1403 2017 22 5 15
si

Failed:  1403 2017 25 5 23
single positional indexer is out-of-bounds
Failed:  1403 2017 25 5 24
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 1
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 2
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 3
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 4
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 5
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 6
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 7
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 8
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 9
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 10
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 11
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 12
single positional indexer is out-of-bounds
Failed:  1403 2017 25 6 13
si